In [1]:
import os
import shutil
from PIL import Image, ImageEnhance, ImageFilter
from PIL import ImageOps
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [91]:
def remove_bad_pictures(directory, good = False):
    directory_files = os.listdir(directory)
    text_files = [x for x in directory_files if x.endswith('.txt')]
    subdir = [x for x in directory_files if not x.endswith('.txt')]
    
    for folder in subdir:
        folder_files = os.listdir(directory + '/' + folder)
        for text_file in text_files:
            with open(directory + '/' + text_file, 'r') as f:
                owls_list = f.readlines()
        
            owls_list_str = ['{:03d}'.format(int(x[:-1])) for x in owls_list]
#             print(bad_owls_list_str)
            
            page_snip = text_file[0:-4]
#             print(page_snip)
            
            page_files_list = [x for x in folder_files if page_snip in x]
#             print(page_files_list)
            
            if good:
                good_files_list = [x for x in page_files_list if any('Image_{}'.format(y) in x
                                                    for y in owls_list_str)]
            else:
                good_files_list = [x for x in page_files_list if not any('Image_{}'.format(y) in x
                                                                     for y in owls_list_str)]
#             if good_files_list == []:
#                 print('Empty: {}'.format(page_snip))
            
            for image in good_files_list:
                new_path = '../data/raw/actual_owls/' + image
                shutil.move(directory + '/' + folder + '/' + image, new_path)
            
    return

In [90]:
# directory = '../data/raw/adobe_drawings'
# remove_bad_pictures(directory)

In [92]:
# directory = '../data/raw/vectorstock'
# remove_bad_pictures(directory, good = True)

In [93]:
# directory = '../data/raw/adobe_sketch'
# remove_bad_pictures(directory, good = True)

In [94]:
# directory = '../data/raw/fineartamerica_digital'
# remove_bad_pictures(directory, good = True)

In [96]:
# directory = '../data/raw/fineartamerica_drawings'
# remove_bad_pictures(directory, good = True)

In [97]:
# directory = '../data/raw/fineartamerica_paintings'
# remove_bad_pictures(directory, good = True)

In [99]:
# from difPy import dif

In [71]:
def crop_fineart_watermarks(directory):
    owls_list = os.listdir(directory)
    fineart_owls = [x for x in owls_list if 'FineArtAmerica' in x]
    other_owls = [x for x in owls_list if x not in fineart_owls]
    end_dir = '../data/raw/actual_owls/unaltered'
    
    for image in fineart_owls:
        img = Image.open(directory + '/' + image)
        img_array = np.array(img)
        img = img.crop((0, 0, img_array.shape[1], 0.8 * img_array.shape[0]))
        img.save(end_dir + '/' + image)
        
    for image in other_owls:
        shutil.copy(directory + '/' + image, end_dir + '/' + image)
    
    return

In [72]:
# directory = '../data/raw/actual_owls/named'
# crop_fineart_watermarks(directory)

In [73]:
def rename_owl_files(directory):
    unaltered_owls = os.listdir(directory)
    unaltered_owls_renamed = ['Owl_Unaltered_{:04d}.jpg'.format(x) for x in range(1, len(unaltered_owls) + 1)]
    
    for i in range(len(unaltered_owls)):
        os.rename(directory + unaltered_owls[i], directory + unaltered_owls_renamed[i])
        
    return

In [75]:
# rename_owl_files('../data/raw/actual_owls/unaltered/')

In [77]:
def convert_to_grayscale(init_dir, end_dir):
    init_dir_files = sorted(os.listdir(init_dir))
    end_dir_files = ['Owl_Grayscale_{}.jpg'.format(x[-8:-4]) for x in init_dir_files]
    
    for index, image in enumerate(init_dir_files):
        img = Image.open(init_dir + '/' + image).convert('L')
        img.save(end_dir + '/' + end_dir_files[index])
        
    return

In [78]:
# init_dir = '../data/raw/actual_owls/unaltered'
# end_dir = '../data/raw/actual_owls/grayscale'
# convert_to_grayscale(init_dir, end_dir)

In [89]:
def resize_image(image_path, size):
    im = Image.open(image_path)
    old_size = im.size
    
    ratio = float(max(size))/max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    
    im = im.resize(new_size, Image.LANCZOS)
    
    new_im = Image.new("L", (size))
    new_im.paste(im, ((size[0] - new_size[0]) // 2, (size[1] - new_size[1]) // 2))
    
    delta_w = size[0] - new_size[0]
    delta_h = size[1] - new_size[1]
    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
    new_im = ImageOps.expand(im, padding)
    
    return new_im

In [191]:
def resize_images(directory, size):
    images = sorted(os.listdir(directory))
    end_dir = '../data/raw/actual_owls/resized'
    end_dir_files = ['Owl_Resized_{}.jpg'.format(x[-8:-4]) for x in images]
    
    for index, image in enumerate(images):
        im = resize_image(directory + '/' + image, size)
        im.save(end_dir + '/' + end_dir_files[index])
        
    return

In [192]:
# directory = '../data/raw/actual_owls/grayscale'
# resize_images(directory, (256, 256))

In [194]:
def invert_images(directory):
    images = sorted(os.listdir(directory))
    end_dir = '../data/raw/actual_owls/inverted'
    end_dir_files = ['Owl_Inverted_{}.jpg'.format(x[-8:-4]) for x in images]
    
    for index, image in enumerate(images):
        im = ImageOps.invert(Image.open(directory + '/' + image))
        im.save(end_dir + '/' + end_dir_files[index])
        
    return

In [196]:
# directory = '../data/raw/actual_owls/resized'
# invert_images(directory)

In [2]:
# Difference of Gaussians applied to img input
def dog(img,size=(0,0),k=1.6,sigma=0.5,gamma=1):
    img1 = cv2.GaussianBlur(img,size,sigma)
    img2 = cv2.GaussianBlur(img,size,sigma*k)
    return (img1-gamma*img2)

# Threshold the dog image, with dog(sigma,k) > 0 ? 1(255):0(0)
def edge_dog(img,sigma=0.5,k=200,gamma=0.98):
    aux = dog(img,sigma=sigma,k=k,gamma=0.98)
    for i in range(0,aux.shape[0]):
        for j in range(0,aux.shape[1]):
            if(aux[i,j] > 0):
                aux[i,j] = 255
            else:
                aux[i,j] = 0
    return aux

# garygrossi xdog version
def xdog_garygrossi(img,sigma=0.5,k=200, gamma=0.98,epsilon=0.1,phi=10):
    aux = dog(img,sigma=sigma,k=k,gamma=gamma)/255
    for i in range(0,aux.shape[0]):
        for j in range(0,aux.shape[1]):
            if(aux[i,j] >= epsilon):
                aux[i,j] = 1
            else:
                ht = np.tanh(phi*(aux[i][j] - epsilon))
                aux[i][j] = 1 + ht
    return aux*255

In [11]:
def find_percent_whitespace(sketch_arr, white_space_val = 245, fill_space_val = 10):
    
    white_space = (sketch_arr > white_space_val).sum()
    fill_space = (sketch_arr < fill_space_val).sum()
    sketch_size = sketch_arr.shape[0] * sketch_arr.shape[1]
    
    white_space_per = white_space / sketch_size
    fill_space_per = fill_space / sketch_size
    
    return white_space_per, fill_space_per

In [12]:
def find_sketch_threshold(image, white_space_val = 245, fill_space_val = 10,
                            white_space_threshold = 0.5, fill_space_threshold = 0.03):
    
    sketch_thresholds = np.array(list(range(20, 81))) / 100
    
    sketch_to_return = None
    thresh_to_return = 0
    white_space_per_to_return = 100
    fill_space_per_to_return = 0
    
    for thresh in sketch_thresholds:      
        print('Testing gamma {:0.2f}'.format(thresh))
        sketch_arr = np.uint8(xdog_garygrossi(np.array(image), sigma = 0.5, k = 200,
                                              gamma = thresh, epsilon = 0.1, phi = 10))
        white_space_per, fill_space_per = find_percent_whitespace(sketch_arr,
                                                                  white_space_val, fill_space_val)
        if fill_space_per > fill_space_threshold:
            print('Too much fill space, terminating.')
            break
        elif white_space_per < white_space_threshold:
            print('Too little white space, terminating.')
            break  
        sketch_to_return = sketch_arr
        white_space_per_to_return = white_space_per
        fill_space_per_to_return = fill_space_per
        thresh_to_return = thresh
        print('White space: {:02f}, Fill space: {:02f}'.format(white_space_per_to_return, fill_space_per_to_return))
        
        
    if sketch_to_return is not None:    
        sketch = Image.fromarray(sketch_to_return)
    else:
        sketch = None
    
    return sketch, thresh_to_return, white_space_per_to_return, fill_space_per_to_return

In [13]:
def create_sketches(directory, dest_tail, white_space_val = 245, fill_space_val = 10,
                    white_space_threshold = 0.5, fill_space_threshold = 0.03):
    
    images = sorted(os.listdir(directory))
    end_dir = '../data/raw/actual_owls/sketched/{}'.format(dest_tail)
    end_dir_files = ['Owl_Sketched_{}_{}.jpg'.format(dest_tail.title(), x[-8:-4]) for x in images]
    
    for index, image in enumerate(images):
        im = Image.open(directory + '/' + image)
        image_num = image[-8:-4]
        print('Finding best sketch for {}'.format(image_num))
        sketch, thresh, white_space_per, fill_space_per = \
                find_sketch_threshold(im, white_space_val = white_space_val, fill_space_val = fill_space_val,
                                      white_space_threshold = white_space_threshold,
                                      fill_space_threshold = fill_space_threshold)
        
        info_string = 'Image: {}, Threshold: {:0.2f}, White Space: %{:0.2f}, Fill Space: %{:0.2f}'\
                                .format(image_num, thresh, white_space_per, fill_space_per)
        
        if sketch is not None:
            print('Saving sketch for {}'.format(image_num))
            sketch.save(end_dir + '/' + end_dir_files[index])
        else:
            print('No suitable sketch found for {}'.format(image_num))
        
        with open('../data/raw/actual_owls/sketched/{}_sketch_info.txt'.format(dest_tail), 'a') as f:
                f.write(info_string)

    return

In [ ]:
directory = '../data/raw/actual_owls/grayscale'
create_sketches(directory, dest_tail = 'regular')

Finding best sketch for 0001
Testing gamma 0.20
White space: 0.648877, Fill space: 0.000000
Testing gamma 0.21
White space: 0.641806, Fill space: 0.000000
Testing gamma 0.22
White space: 0.636042, Fill space: 0.000000
Testing gamma 0.23
White space: 0.630116, Fill space: 0.000000
Testing gamma 0.24
White space: 0.624711, Fill space: 0.000000
Testing gamma 0.25
White space: 0.618056, Fill space: 0.000000
Testing gamma 0.26
White space: 0.612662, Fill space: 0.000000
Testing gamma 0.27
White space: 0.606424, Fill space: 0.000000
Testing gamma 0.28
White space: 0.600382, Fill space: 0.000000
Testing gamma 0.29
White space: 0.594456, Fill space: 0.000000
Testing gamma 0.30
White space: 0.588588, Fill space: 0.000000
Testing gamma 0.31
White space: 0.582280, Fill space: 0.000000
Testing gamma 0.32
White space: 0.576516, Fill space: 0.000000
Testing gamma 0.33
White space: 0.569525, Fill space: 0.000000
Testing gamma 0.34
White space: 0.564329, Fill space: 0.000000
Testing gamma 0.35
White s

White space: 0.810170, Fill space: 0.022207
Testing gamma 0.25
Too much fill space, terminating.
Saving sketch for 0008
Finding best sketch for 0009
Testing gamma 0.20
White space: 0.898056, Fill space: 0.000000
Testing gamma 0.21
White space: 0.890856, Fill space: 0.000015
Testing gamma 0.22
White space: 0.884660, Fill space: 0.000085
Testing gamma 0.23
White space: 0.880725, Fill space: 0.000116
Testing gamma 0.24
White space: 0.877400, Fill space: 0.000409
Testing gamma 0.25
White space: 0.873873, Fill space: 0.000725
Testing gamma 0.26
White space: 0.870347, Fill space: 0.001204
Testing gamma 0.27
White space: 0.865517, Fill space: 0.001806
Testing gamma 0.28
White space: 0.859660, Fill space: 0.002554
Testing gamma 0.29
White space: 0.852670, Fill space: 0.003187
Testing gamma 0.30
White space: 0.845378, Fill space: 0.004174
Testing gamma 0.31
White space: 0.838333, Fill space: 0.005556
Testing gamma 0.32
White space: 0.830772, Fill space: 0.007353
Testing gamma 0.33
White space: 

White space: 0.919753, Fill space: 0.028410
Testing gamma 0.42
Too much fill space, terminating.
Saving sketch for 0016
Finding best sketch for 0017
Testing gamma 0.20
White space: 0.884500, Fill space: 0.004562
Testing gamma 0.21
White space: 0.879385, Fill space: 0.006271
Testing gamma 0.22
White space: 0.874469, Fill space: 0.008448
Testing gamma 0.23
White space: 0.869333, Fill space: 0.010729
Testing gamma 0.24
White space: 0.864448, Fill space: 0.012958
Testing gamma 0.25
White space: 0.859313, Fill space: 0.015479
Testing gamma 0.26
White space: 0.854615, Fill space: 0.017708
Testing gamma 0.27
White space: 0.849042, Fill space: 0.020552
Testing gamma 0.28
White space: 0.843771, Fill space: 0.023208
Testing gamma 0.29
White space: 0.838427, Fill space: 0.025667
Testing gamma 0.30
White space: 0.833104, Fill space: 0.028698
Testing gamma 0.31
Too much fill space, terminating.
Saving sketch for 0017
Finding best sketch for 0018
Testing gamma 0.20
White space: 0.938657, Fill space:

White space: 0.952255, Fill space: 0.005104
Testing gamma 0.26
White space: 0.950495, Fill space: 0.006083
Testing gamma 0.27
White space: 0.948734, Fill space: 0.007068
Testing gamma 0.28
White space: 0.947120, Fill space: 0.008021
Testing gamma 0.29
White space: 0.945109, Fill space: 0.008937
Testing gamma 0.30
White space: 0.943203, Fill space: 0.010083
Testing gamma 0.31
White space: 0.941302, Fill space: 0.010937
Testing gamma 0.32
White space: 0.939036, Fill space: 0.011984
Testing gamma 0.33
White space: 0.937068, Fill space: 0.013036
Testing gamma 0.34
White space: 0.935427, Fill space: 0.013932
Testing gamma 0.35
White space: 0.933109, Fill space: 0.015198
Testing gamma 0.36
White space: 0.931120, Fill space: 0.016385
Testing gamma 0.37
White space: 0.929068, Fill space: 0.017641
Testing gamma 0.38
White space: 0.927094, Fill space: 0.018917
Testing gamma 0.39
White space: 0.925125, Fill space: 0.020464
Testing gamma 0.40
White space: 0.922984, Fill space: 0.021943
Testing gam

White space: 0.995003, Fill space: 0.000000
Testing gamma 0.25
White space: 0.994420, Fill space: 0.000000
Testing gamma 0.26
White space: 0.993765, Fill space: 0.000000
Testing gamma 0.27
White space: 0.993193, Fill space: 0.000000
Testing gamma 0.28
White space: 0.992650, Fill space: 0.000000
Testing gamma 0.29
White space: 0.992088, Fill space: 0.000005
Testing gamma 0.30
White space: 0.991571, Fill space: 0.000005
Testing gamma 0.31
White space: 0.991060, Fill space: 0.000015
Testing gamma 0.32
White space: 0.990497, Fill space: 0.000026
Testing gamma 0.33
White space: 0.989786, Fill space: 0.000046
Testing gamma 0.34
White space: 0.989178, Fill space: 0.000066
Testing gamma 0.35
White space: 0.988620, Fill space: 0.000087
Testing gamma 0.36
White space: 0.988032, Fill space: 0.000123
Testing gamma 0.37
White space: 0.987331, Fill space: 0.000169
Testing gamma 0.38
White space: 0.986661, Fill space: 0.000210
Testing gamma 0.39
White space: 0.986022, Fill space: 0.000261
Testing gam

...

White space: 0.961188, Fill space: 0.012145
Testing gamma 0.41
White space: 0.959992, Fill space: 0.013356
Testing gamma 0.42
White space: 0.958997, Fill space: 0.014668
Testing gamma 0.43
White space: 0.957955, Fill space: 0.015833
Testing gamma 0.44
White space: 0.956597, Fill space: 0.016906
Testing gamma 0.45
White space: 0.955455, Fill space: 0.017731
Testing gamma 0.46
White space: 0.954159, Fill space: 0.018603
Testing gamma 0.47
White space: 0.952639, Fill space: 0.019568
Testing gamma 0.48
White space: 0.951096, Fill space: 0.020556
Testing gamma 0.49
White space: 0.948997, Fill space: 0.021335
Testing gamma 0.50
White space: 0.946690, Fill space: 0.022137
Testing gamma 0.51
White space: 0.944367, Fill space: 0.022901
Testing gamma 0.52
White space: 0.941551, Fill space: 0.023827
Testing gamma 0.53
White space: 0.938465, Fill space: 0.024900
Testing gamma 0.54
White space: 0.935409, Fill space: 0.026080
Testing gamma 0.55
White space: 0.931844, Fill space: 0.027500
Testing gam

White space: 0.983803, Fill space: 0.000206
Testing gamma 0.30
White space: 0.982777, Fill space: 0.000356
Testing gamma 0.31
White space: 0.981503, Fill space: 0.000511
Testing gamma 0.32
White space: 0.980219, Fill space: 0.000731
Testing gamma 0.33
White space: 0.978907, Fill space: 0.000900
Testing gamma 0.34
White space: 0.977605, Fill space: 0.001134
Testing gamma 0.35
White space: 0.976171, Fill space: 0.001448
Testing gamma 0.36
White space: 0.974606, Fill space: 0.001865
Testing gamma 0.37
White space: 0.973220, Fill space: 0.002399
Testing gamma 0.38
White space: 0.971772, Fill space: 0.002928
Testing gamma 0.39
White space: 0.970067, Fill space: 0.003462
Testing gamma 0.40
White space: 0.968530, Fill space: 0.004081
Testing gamma 0.41
White space: 0.966778, Fill space: 0.004784
Testing gamma 0.42
White space: 0.965007, Fill space: 0.005528
Testing gamma 0.43
White space: 0.963095, Fill space: 0.006334
Testing gamma 0.44
White space: 0.961371, Fill space: 0.007061
Testing gam

White space: 0.994244, Fill space: 0.000000
Testing gamma 0.30
White space: 0.993395, Fill space: 0.000000
Testing gamma 0.31
White space: 0.992461, Fill space: 0.000000
Testing gamma 0.32
White space: 0.991551, Fill space: 0.000000
Testing gamma 0.33
White space: 0.990471, Fill space: 0.000000
Testing gamma 0.34
White space: 0.989452, Fill space: 0.000000
Testing gamma 0.35
White space: 0.988395, Fill space: 0.000000
Testing gamma 0.36
White space: 0.987323, Fill space: 0.000023
Testing gamma 0.37
White space: 0.986119, Fill space: 0.000023
Testing gamma 0.38
White space: 0.984892, Fill space: 0.000039
Testing gamma 0.39
White space: 0.983488, Fill space: 0.000069
Testing gamma 0.40
White space: 0.982091, Fill space: 0.000131
Testing gamma 0.41
White space: 0.980571, Fill space: 0.000216
Testing gamma 0.42
White space: 0.978611, Fill space: 0.000463
Testing gamma 0.43
White space: 0.977114, Fill space: 0.000895
Testing gamma 0.44
White space: 0.975278, Fill space: 0.001481
Testing gam

Too much fill space, terminating.
Saving sketch for 0061
Finding best sketch for 0062
Testing gamma 0.20
White space: 0.909552, Fill space: 0.000177
Testing gamma 0.21
White space: 0.905323, Fill space: 0.000375
Testing gamma 0.22
White space: 0.901417, Fill space: 0.000760
Testing gamma 0.23
White space: 0.897656, Fill space: 0.001208
Testing gamma 0.24
White space: 0.893375, Fill space: 0.001781
Testing gamma 0.25
White space: 0.888948, Fill space: 0.002594
Testing gamma 0.26
White space: 0.884802, Fill space: 0.003417
Testing gamma 0.27
White space: 0.880927, Fill space: 0.004354
Testing gamma 0.28
White space: 0.876708, Fill space: 0.005615
Testing gamma 0.29
White space: 0.872698, Fill space: 0.006844
Testing gamma 0.30
White space: 0.868490, Fill space: 0.008104
Testing gamma 0.31
White space: 0.864208, Fill space: 0.009479
Testing gamma 0.32
White space: 0.859865, Fill space: 0.011240
Testing gamma 0.33
White space: 0.855052, Fill space: 0.013104
Testing gamma 0.34
White space: 

White space: 0.948557, Fill space: 0.000013
Testing gamma 0.25
White space: 0.943060, Fill space: 0.000013
Testing gamma 0.26
White space: 0.938328, Fill space: 0.000038
Testing gamma 0.27
White space: 0.933358, Fill space: 0.000100
Testing gamma 0.28
White space: 0.928075, Fill space: 0.000176
Testing gamma 0.29
White space: 0.922540, Fill space: 0.000264
Testing gamma 0.30
White space: 0.917194, Fill space: 0.000351
Testing gamma 0.31
White space: 0.912048, Fill space: 0.000414
Testing gamma 0.32
White space: 0.907267, Fill space: 0.000577
Testing gamma 0.33
White space: 0.902108, Fill space: 0.000728
Testing gamma 0.34
White space: 0.897176, Fill space: 0.000979
Testing gamma 0.35
White space: 0.892194, Fill space: 0.001293
Testing gamma 0.36
White space: 0.886358, Fill space: 0.001682
Testing gamma 0.37
White space: 0.881300, Fill space: 0.002196
Testing gamma 0.38
White space: 0.876657, Fill space: 0.002836
Testing gamma 0.39
White space: 0.871398, Fill space: 0.003677
Testing gam

In [ ]:
directory = '../data/raw/actual_owls/inverted'
create_sketches(directory, dest_tail = 'inverted')